# 1. Extracción de datos de la API de USGS

In [42]:
import requests
import pandas as pd

In [43]:
# los paises y los estados (USA) que se buscan filtrar
paises = ['Chile', 'Japan']
us_estados = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
             'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
             'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 
             'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 
             'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
             'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 
             'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 
             'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 
             'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [44]:
def find_country(place):
    # busca en la lista de paises
    for pais in paises:
        if pais.lower() in place.lower():
            return pais.lower()
    # busca en la lista de estados de Estados Unidos
    for estado in us_estados:
        if estado.lower() in place.lower():
            return 'usa'
    return 'other'  # devuelve 'other' si no encuentra una coincidencia

In [45]:
#se establece la url y los parámetros de la request
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "geojson",
    "starttime": "2014-01-01",
    "endtime": "2014-01-31"
}
response = requests.get(url, params=params)

# se guardan los datos de la consulta a la API
data = response.json()


In [46]:
# se toman los datos del diccionario correspondiente
features = data['features']

In [47]:
filtered_features = []

for feature in features:
    place = feature['properties']['place']
    for location in paises + us_estados:  # se verifica tanto en países como en estados de EE.UU.
        if location.lower() in place.lower():
            filtered_features.append(feature)
            break  # se termina la busqueda buscar en los otros lugares una vez que encontramos una coincidencia

# Ahora 'filtered_features' contiene solo los terremotos que ocurrieron en los países y estados especificados


In [48]:
len(filtered_features) #se revisa la cantidad de registros filtrados

8087

In [49]:
# Crear una lista vacía para almacenar los datos
data_list = []

for feature in filtered_features:
    # Extraer información de properties y geometry
    properties = feature['properties']
    geometry = feature['geometry']

    # Crear un diccionario con los datos que necesitamos
    data_dict = {
        'place': properties['place'],
        'mag': properties['mag'],
        'time': pd.to_datetime(properties['time'], unit='ms'),  # convertir el tiempo a formato legible
        'lon': geometry['coordinates'][0],
        'lat': geometry['coordinates'][1],
        'depth': geometry['coordinates'][2]
    }

    # Añadir el diccionario a la lista
    data_list.append(data_dict)

# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(data_list)


In [50]:
df.shape #dimensiones del df

(8087, 6)

In [52]:
# se crea la columna con la denominación de cada país según la columna place
df['country'] = df['place'].apply(find_country)

In [53]:
df.head()

,place,mag,time,lon,lat,depth,country
0,"4 km WNW of Cobb, California",0.45,2014-01-30 23:59:35.960,-122.774666,38.837666,1.960,usa
1,"47 km N of Petersville, Alaska",2.70,2014-01-30 23:40:17.942,-150.650200,62.918300,108.600,usa
2,Northern California,0.43,2014-01-30 23:21:12.670,-122.801834,38.837666,1.659,usa
3,"28 km E of Pedro Bay, Alaska",1.90,2014-01-30 23:15:34.167,-153.599800,59.763900,125.100,usa
4,"5 km S of Volcano, Hawaii",1.71,2014-01-30 23:14:27.970,-155.240000,19.396000,27.743,usa
